### The objective of this work is to creat a stratigraphy data with name, its depth and its respective measured distance (md)  and true vertical distance(tvd). To do this two json file are presented.  
              1: stratigraphy.json 
              2: wellpath.json 
      
  

In [1]:
pwd

'C:\\Users\\joshi\\Downloads\\Trial Pwp'

In [ ]:
# import the neccsary python libraries 
import pandas as pd
import numpy as np 
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

####  Import the json files in to pandas dataFram

In [ ]:
df_stratigraphy = pd.read_json('stratigraphy.json')

In [ ]:
df_stratigraphy.head(2)

In [ ]:
df_stratigraphy.info()

In [ ]:
df_stratigraphy['md'] = df_stratigraphy.pickDepth*0.305800 # change the pickDepth column to meter 

In [ ]:
df_stratigraphy.head(2)

In [ ]:
df_stratigraphy_2 = df_stratigraphy.round()

In [ ]:
df_stratigraphy_2.head(2)

In [ ]:
df_stratigraphy_2.columns 

In [ ]:
df_well = pd.read_json('wellpath.json') # load a json file 

In [ ]:
df_well_2=df_well.round(2) # round to two decimal place 

In [ ]:
merge_md = pd.merge(df_stratigraphy_2, df_well_2,  on='md')# merge the two dataframe in one by measured distance(md)

In [ ]:
merge_md.head() # 

In [ ]:
merge_md.info() # the merged file information

In [ ]:
merge_md_sorted  = merge_md.sort_values(by='md')  # sort the date by measured distance (md)

In [ ]:
 print(merge_md_sorted.head())

In [ ]:
final_well = merge_md_sorted[['pickName', 'md', 'tvd']] # select the relevent columns 

In [ ]:
final_well['depth'] = np.where(final_well.pickName.str.contains('top'),'top','bottom')# add a new depth column 
final_well

In [ ]:
final=final_well.to_dict(orient = 'records')
final

In [ ]:
#alternate depth top and bottom and later we rearrange again based on the value of tvd
keyList=kl=[i['depth'] for i in final]
for k in range(len(keyList)-1):
    if keyList[k]==keyList[k+1]:
        if keyList[k]=='top':
            keyList[k+1]='bottom'
        elif keyList[k]=='bottom':
            keyList[k+1]='top'        
keyList  

In [ ]:
final_well['depth']=keyList
print(final_well)

In [ ]:
well_arr = final_well.reindex(columns= ['pickName','depth', 'md', 'tvd']) # rearrange the order of the columns 
print(well_arr)


In [ ]:
dic=well_arr.to_dict(orient = 'records')
dic

In [ ]:
"""create a nested dictionary"""
def func1(d):
    final=list()
    for dic in d:
        temp=dict()
        temp2=dict()
        temp['pickName']=dic['pickName']
        temp['depth']=dic['depth']
        if temp['depth']=='bottom':            
            temp2['bottom']={key:dic[key] for key in list(dic)[2:] }
            temp['depth']={'bottom':temp2['bottom']}        
        else:
            temp2['top']={key:dic[key] for key in list(dic)[2:] }
            temp['depth']={'top':temp2['top']}                
        final.append(temp)
    return final


In [ ]:
final=func1(dic)
pp.pprint(final)

In [ ]:
pd.DataFrame.from_dict(final)

In [ ]:
""" make the depth of the current marker is equal to the bottom depth of the previous marker"""
def func(final):
    cc=list()
    for i in range(len(final)-1):       
        c=dict()
        if final[i]['pickName'].endswith('top'):
            c['pickName']=final[i]['pickName']
            # Merge current dic md and tvd value to dic below it
            c['depth']={**final[i]['depth'], **final[i+1]['depth']}
            cc.append(c)
        elif final[i]['pickName'].endswith('base') or final[i]['pickName'].endswith('ALT'):
            c['pickName']=final[i]['pickName']
            c['depth']={**final[i]['depth'], **final[i+1]['depth']}
            # Interchange bottom values to top and top values to bottom 
            val=list(c['depth'].values())
            c['depth']['bottom']=val[1]
            c['depth']['top']=val[0]
            cc.append(c)      
    return cc


In [ ]:
resp=func(final)
pp.pprint(resp)
#print(resp)

In [ ]:
"""the code arrange the top and bottom on pickName H_base, which the top is larger than the bottom """
def arrangeTopAndBottom(dic):    
    for i in range(len(dic)):
        if dic[i]['depth']['bottom']['tvd']<dic[i]['depth']['top']['tvd']:
                val=list(dic[i]['depth'].values())
                #print(val)
                dic[i]['depth']['bottom']=val[1]
                dic[i]['depth']['top']=val[0] 
    return dic
    


In [ ]:
diff = arrangeTopAndBottom(resp)
#pp.pprint(arr_list)
print(diff)

In [ ]:
len(diff)

In [ ]:
"""changing the name of the bases to undifferentiated"""
for d in range(len(diff)):
    if 'base' in diff[d]['pickName']:
        diff[d]['pickName'] = 'undifferentiated'
        #print(list(diff[d].keys()))

In [ ]:
diff